In [2]:
from typing import Any, List, Dict, Iterable
import os
import wandb

api = wandb.Api(timeout=19)

# run is specified by <entity>/<project>/<run id>
# run = api.run("quartz/PPO-Finetune-03/2bjnzv3w")

# save the metrics for the run to a csv file
# metrics_dataframe = run.scan_history() # keys=['_step', '_timestamp', 'iter'])

In [3]:
def wandb_link_to_path(link: str):
    if link.startswith('https://') or link.startswith('https://'):
        # https://wandb.ai/quartz/PPO-Finetune-03/runs/2bjnzv3w
        _path: List[str] = link.split('//')[1].split('/')
        run_path: str = '/'.join([_path[1], _path[2], _path[4]])
    else:
        run_path = link
    return run_path

def circ_name_from_run(run):
    return run.config['_content']['input_graphs'][0].split("'")[3]

def best_info(run_path: str, max_runtime: float = 24*3600):
    run_path = wandb_link_to_path(run_path)
    run = api.run(run_path)
    it_metric_dicts: Iterable[Dict[str, Any]] = run.scan_history()

    circ_name = None
    best_keys = ['best_graph_cost', 'best_graph_cx_count', 'best_graph_depth', 'best_graph_gate_count']
    best_costs = {
        k: (-1, -1, -1) # (value, iter, runtime)
        for k in best_keys
    }
    i_dict: int = -1
    for metric_dict in it_metric_dicts:
        if not circ_name: # the first iteration
            i_dict = 0
            keys: List[str] = list(metric_dict.keys())
            best_cost_key: str = list(filter(lambda x: x.endswith('best_graph_cost'), keys))[0]
            circ_name = best_cost_key.split('_best_graph_cost')[0]
        # get best info
        if f'{circ_name}_{best_keys[0]}' not in metric_dict:
            i_dict += 1
            continue
        if metric_dict['_runtime'] > max_runtime:
            break
        for k in best_costs:
            bvalue, biter, bruntime = best_costs[k]
            value_cand = metric_dict[f'{circ_name}_{k}']
            if bvalue < 0 or value_cand < bvalue:
                # update!
                best_costs[k] = (value_cand, metric_dict['iter'], metric_dict['_runtime'])
        # end for k
        i_dict += 1
    # end for
    assert circ_name, f'{circ_name = }'
    return circ_name, best_costs
    
# best_info('https://wandb.ai/quartz/PPO-Finetune-03/runs/2bjnzv3w')

In [4]:
with open('nam_worm_wandb.txt') as f:
    _f = f.read()
links: List[str] = _f.split()
print(f'{len(links) = }')

# verify first
for i_link, link in enumerate(links):
    run = api.run(wandb_link_to_path(link))
    circ_name = run.config['_content']['input_graphs'][0].split("'")[3]
    print(f'{i_link+1}: {circ_name}')

for link in links:
    circ_name, best_costs = best_info(link)
    best_info_str = ''
    # for k, t in best_costs.items():
    for k in ['best_graph_gate_count', 'best_graph_cx_count', 'best_graph_depth']:
        t = best_costs[k]
        best_info_str += f'{t[0]}\t{t[1]}\t{t[2]:.0f}\t'
    print(f'{circ_name}\t\t{best_info_str}')


len(links) = 27
1: adder_8
2: barenco_tof_3
3: barenco_tof_4
4: barenco_tof_5
5: barenco_tof_10
6: csla_mux_3
7: csum_mux_9
8: gf2^4_mult
9: gf2^5_mult
10: gf2^6_mult
11: gf2^7_mult
12: gf2^8_mult
13: gf2^9_mult
14: gf2^10_mult
15: gf2^16_mult
16: mod5_4
17: mod_mult_55
18: mod_red_21
19: qcla_adder_10
20: qcla_com_7
21: qcla_mod_7
22: rc_adder_6
23: tof_3
24: tof_4
25: tof_5
26: tof_10
27: vbe_adder_3
adder_8		556	415	84448	255	415	84448	175	292	45446	
barenco_tof_3		35	187	25084	13	187	25084	32	187	25084	
barenco_tof_4		62	192	30539	24	192	30539	56	192	30539	
barenco_tof_5		91	296	81088	37	296	81088	83	296	81088	
barenco_tof_10		230	427	59579	96	427	59579	204	427	59579	
csla_mux_3		140	177	9445	63	176	9305	57	176	9305	
csum_mux_9		334	349	44306	138	349	44306	61	103	29	
gf2^4_mult		161	326	32247	80	326	32247	84	326	32247	
gf2^5_mult		254	564	77875	129	564	77875	112	199	14319	
gf2^6_mult		366	481	75512	189	481	75512	172	158	8324	
gf2^7_mult		495	451	82712	253	451	82712	197	451	82712	
g